In [1]:
##### Se estiver executando no COLAB rode esta linha e restart o Kernel em seguida
#!python -m spacy download pt_core_news_sm

In [2]:
#Se tiver exwecutando no Colab rode esta célula
#from google.colab import drive
#drive.mount('/content/gdrive')
#import sys
#sys.path.insert(0,'/content/gdrive/My Drive/TCC/tcccimatec')
#caminhoDados="/content/gdrive/My Drive/TCC/tcccimatec/dados/"


In [3]:
#Dados disponíveis em 
#https://drive.google.com/drive/folders/111ViaBRVmR0uevl00Ivmy3tbqMLjqIX7?usp=sharing

In [1]:
#Se estiver rodando localmente 
caminhoDados="dados/"

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from zipfile import ZipFile
import nltk
from nltk.corpus import stopwords
import re
from unicodedata import normalize
from nltk.tokenize import RegexpTokenizer, WordPunctTokenizer, WhitespaceTokenizer
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import html
import spacy
from libs import stopword, tratatexto
nlp = spacy.load('pt_core_news_sm')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
random_state=26

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/94779325587/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/94779325587/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/94779325587/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Análise Exploratória

In [3]:
df  = pd.read_json(caminhoDados+'segmentosOrigemAnotacao.json') 
#Excluindo o ano de 2021  por não estar completo
df = df[df['ano']<2021]
#unificando o nome da coluna idSegmento
df.rename(columns={'idSegmentoOrigem': 'idSegmento'},inplace=True)


In [4]:
len(df.index)

21012

In [5]:
df['ano'].unique()

array([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020])

In [8]:
#Quantidade de segmentos anotados por tipo de Solução
#Percebemos que alguns segmentos tem mais de uma anotação, esses casos não serão escopo desse trabalho
#o alvo são segmentos que só possuem uma anotação
df.groupby(["idSegmento"])["nomeTipoAnotacao"].nunique().sort_values(ascending=False)
dfseg = pd.DataFrame(df.groupby(["idSegmento"])["nomeTipoAnotacao"].nunique())
excluir = dfseg[dfseg['nomeTipoAnotacao']>1].index
#Quantidade de segmentos a exluir
len(excluir)
df.drop(df[df['idSegmento'].isin(excluir)].index, inplace = True)

In [9]:
#As 28 anotacoes sem nome são anotações válidas para o negócio, no sistema tem um valor "Inclusão por republicação parcial"
df.loc[df['nomeTipoAnotacao']=="", 'nomeTipoAnotacao']= "Inclusão por republicação parcial"

In [10]:
dfAlvoAnota = df

In [11]:
idAtoAlvo = dfAlvoAnota['idAto'].unique()

# Tratamento texto

In [12]:
#Texto que tem anotação

In [13]:
dfAlvoAnota['ementalimpa'] = dfAlvoAnota['ementa'].apply(tratatexto.limpaTexto)

In [14]:
dfAlvoAnota['verbosEmenta'] = dfAlvoAnota['ementalimpa'].apply(tratatexto.buscaVerbo)

In [15]:
dfAlvoAnota['textoIntegraLimpo'] = dfAlvoAnota['textoIntegra'].apply(tratatexto.limpaTexto)

In [16]:
dfAlvoAnota['textoIntegraLimpo'][0]

'art autorizacao tratar art ficar condicionar comprovacao referir art incisar instrucao normativo '

In [17]:
dfAlvoAnota['verbosTextoIntegra'] = dfAlvoAnota['textoIntegraLimpo'].apply(tratatexto.buscaVerbo)

# Gerando Json

In [18]:

#Para os segmentos que não anotam não faz sentido usar a ementa nem seus verbos
valores = {'idTipoAnotacao': 0, 'nomeTipoAnotacao':'Não anota','verbosEmenta':'','ementalimpa':''}
dfAlvoAnota.fillna(value = valores, inplace = True)

#colunas que não importam para o tratamento de textos
#limpar = ['ano','ementa','idAto','idTipoAto','idTipoSegmento','nomeTipoAto','nomeTipoSegmento','siglaTipoAnotacao','tipoSolucao','versaoSegmento']
#dfAlvoAnota.drop(limpar,axis=1, inplace=True)

#Gerando arquivo 
dfAlvoAnota['verbos'] = dfAlvoAnota['verbosEmenta']+ " "  + dfAlvoAnota['verbosTextoIntegra']
dfAlvoAnota['textoTotal'] = dfAlvoAnota['ementalimpa']+ " "  + dfAlvoAnota['textoIntegraLimpo']
dfAlvoAnota.to_json(caminhoDados+'dfAlvoAnota.json')
dfAlvoAnota  = pd.read_json(caminhoDados+'dfAlvoAnota.json') 
dfAlvoAnota.head()

,ano,ementa,idAto,idSegmento,idTipoAnotacao,idTipoAto,idTipoSegmento,nomeTipoAnotacao,nomeTipoAto,nomeTipoSegmento,siglaTipoAnotacao,textoIntegra,tipoSolucao,versaoSegmento,ementalimpa,verbosEmenta,textoIntegraLimpo,verbosTextoIntegra,verbos,textoTotal
0,2013,Autoriza exportação de cigarros do estabelecim...,39246,1260061,24,9,3,Vide,Ato Declaratório Executivo,Artigo,VID,Art. 2º A autorização de que trata o art. 1º f...,REF,1,autorizar exportacao cigarro estabelecimento e...,autorizar inscrever,art autorizacao tratar art ficar condicionar c...,tratar ficar condicionar referir incisar,autorizar inscrever tratar ficar condicionar r...,autorizar exportacao cigarro estabelecimento e...
1,2013,"Altera a Instrução Normativa SRF nº 588, de 21...",39255,1260829,1,42,999,Alteração,Instrução Normativa,Não Identificado,RedaÃ§Ã£o dada,“Art. 17. As entidades fechadas de previdência...,ALT,1,alterar instrucao normativo srf dezembro dispo...,alterar dispoe beneficiar vidar clausular prov...,art entidade fechar previdencia complementar i...,fechar impor render dever lucrar liquidar,alterar dispoe beneficiar vidar clausular prov...,alterar instrucao normativo srf dezembro dispo...
10,2013,"Altera a Instrução Normativa SRF nº 386, de 14...",39322,1264027,3,42,5,Inclusão,Instrução Normativa,Parágrafo,INC,§ 3º Na hipótese de exigência de controle admi...,INC,1,alterar instrucao normativo srf janeiro dispoe...,alterar dispoe depositar,hipotese exigencia controlo administrativo par...,exigencia partir despachar consumir mercadoria...,alterar dispoe depositar exigencia partir desp...,alterar instrucao normativo srf janeiro dispoe...
100,2013,Dispõe sobre a instituição de códigos de recei...,39584,1271069,9,9,15,Revogação,Ato Declaratório Executivo,Inciso,RVO,"I - o Ato Declaratório Executivo Codac nº 31, ...",EXC,1,dispoe instituicao codigos caso especificar co...,dispoe especificar consolidar indevido,atar declaratorio executivo codac abril,atar codac,dispoe especificar consolidar indevido atar codac,dispoe instituicao codigos caso especificar co...
1000,2013,"Altera a Instrução Normativa nº 1.307, de 27 d...",45387,1334040,1,42,15,Alteração,Instrução Normativa,Inciso,RedaÃ§Ã£o dada,"XII - a Secretaria Especial de Portos, no que ...",ALT,1,alterar instrucao normativo dezembro dispoe de...,alterar dispoe,xii secretariar especial porto respeitar proje...,secretariar respeitar reidi aprovar,alterar dispoe secretariar respeitar reidi apr...,alterar instrucao normativo dezembro dispoe de...
